In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!pip install thop
!pip install ptflops


In [9]:


#change the directory manually if you are not marco or allan
who_is_accesing = input("say your name ")
if who_is_accesing == "marco":
  %cd "/content/drive/MyDrive/Fourth Year/Machine learning and Deep learning Course/task_3"
  directory  = "/content/drive/MyDrive/Fourth Year/Machine learning and Deep learning Course/task_3"
elif who_is_accesing == "allan":
  %cd "/content/drive/MyDrive/Summer 2022/task_3"
  directory = "/content/drive/MyDrive/Summer 2022/task_3"
elif who_is_accesing == "mldl":
  %cd "/content/drive/MyDrive/task_3"
  directory = "/content/drive/MyDrive/task_3"


say your name marco
/content/drive/MyDrive/Fourth Year/Machine learning and Deep learning Course/task_3


In [10]:

import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from utils import poly_lr_scheduler
from utils import reverse_one_hot, compute_global_accuracy, fast_hist, per_class_iu
from torch.autograd import Variable

import torch.cuda.amp as amp
import argparse
import argparse
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import time
#Models
from model.build_BiSeNet import BiSeNet
from model.discriminator import Discriminator
from model.discriminator import DiscriminatorSEP

#losses
from loss import CrossEntropyLoss
from torch.nn import BCEWithLogitsLoss
from loss import DiceLoss
from loss import find_frequency_class
from loss import get_weights_classes_inverse
from loss import calculate_weigths_labels
from loss import FocalLoss

#datasets
from dataset.cityscapes_dataset import CityScapeDataSet
from dataset.gta5_dataset import GTA5DataSet
from dataset.cityscapes_dataset_val import CityScapeDataSet_val



In [11]:
def validation_segmentation(args, modelSEG, dataloader, name_instance):
  print('start validation')
  with torch.no_grad(): 
    modelSEG.eval()
    precision_record = []
    hist = np.zeros((args.num_classes, args.num_classes))

    for i, (image, label) in enumerate(dataloader):
      label = label.type(torch.LongTensor)
      label = label.long().cuda()
      image = image.cuda()

      # get RGB predict image
      predict = modelSEG(image).squeeze()
      
      predict = reverse_one_hot(predict)
     
      predict = predict.cpu().numpy()
      

      # get RGB label image
      label = label.squeeze()
      label = np.array(label.cpu())

      # compute per pixel accuracy
      precision = compute_global_accuracy(predict, label)
      hist += fast_hist(label.flatten(), predict.flatten(), args.num_classes)
      
      precision_record.append(precision)

    precision = np.mean(precision_record)
    miou_list = per_class_iu(hist)
    miou = np.mean(miou_list)
    print('precision per pixel for test: %.3f' % precision)
    print('mIoU for validation: %.3f' % miou)
    print(f'mIoU per class: {miou_list}')

    name_file = "_".join(tuple(name_instance.split("_")[0:5]))

    #Save the results in a file inside the validation folder
    with open(args.val_results_directory + "/" + name_file + ".txt", "w") as pipe:
      pipe.write("Precision per pixel: " + str(precision) + "\n")
      pipe.write("Average mIoU: " + str(miou) + "\n")
      for index, x in enumerate(miou_list):
        pipe.write("Class " + str(index) + ": " + str(x) + "\n")


  return precision, miou

In [12]:
def train_model(args, modelSEG, modelADV, optimizerSEG, optimizerADV, dataloader_target, dataloader_source, model_seg_dictionary = None, model_adv_dictionary = None):
  
  #initialize the dictionaries for the checkpoints
  if model_seg_dictionary != None and model_adv_dictionary != None:
    print("Using checkpoints")
    modelSEG.load_state_dict(model_seg_dictionary['model_state_dict'])
    modelADV.load_state_dict(model_adv_dictionary['model_state_dict'])

    optimizerSEG.load_state_dict(model_seg_dictionary['optimizer_state_dict'])
    optimizerADV.load_state_dict(model_adv_dictionary['optimizer_state_dict'])

    initial_epoch = model_seg_dictionary["epoch"]
    checkpoint = True

  else:
    initial_epoch = 0

  #initializing scaler object
  scaler = amp.GradScaler()

  #set the model to train mode
  modelSEG.train()
  modelADV.train()


  #Setting the losses functions
  if args.weights_loss == "False":
    print("Not using loss weights")
    weights_loss = torch.ones(args.num_classes).cuda()
    weights_loss = weights_loss.float()
  else:
    print("Using loss weights")
    #In here it tries to read the weights from the stored folder,
    # if it fails it will calculate again the loss weights
    try:
      weights_loss = torch.tensor(np.load(os.path.join(args.weights_loss_dir, 'classes_weights.npy')))
      weights_loss = weights_loss.cuda().float()
      print(weights_loss)
    except:
      weights_loss = calculate_weigths_labels(args.weights_loss_dir, dataloader_source, args.num_classes)
      weights_loss = weights_loss.cuda()
      print(weights_loss)
  
  if args.loss_type == "CrossEntropy":
    loss_seg_source = torch.nn.CrossEntropyLoss(weight = weights_loss, ignore_index=255) #loss to train the segmentation network with the source dataset
    loss_adv_target_source = torch.nn.BCEWithLogitsLoss() #loss to train the segmenation network with the target dataset via the adversarial network
  else:
    loss_seg_source = FocalLoss(weight = weights_loss, ignore_index=255) #loss to train the segmentation network with the source dataset
    loss_adv_target_source = torch.nn.BCEWithLogitsLoss() #loss to train the segmenation network with the target dataset via the adversarial network

  loss_D = torch.nn.BCEWithLogitsLoss() #loss to train only the adversarial network

  # labels for adversarial training
  source_label = 0
  target_label = 1

  #Defining the optmizers
  optmizer_segmentation = optimizerSEG
  optmizer_adversarial = optimizerADV

  epoch = initial_epoch
  
  while epoch < args.num_epochs:
    print("Epoch = ",epoch)

    pres = 0
    prev = 0
    #Defining the iterator object for the dataloaders
    dataloader_source_iter = enumerate(dataloader_source)
    dataloader_target_iter = enumerate(dataloader_target)  

    #Update learning rate
    lr_seg = poly_lr_scheduler(optimizerSEG, args.learning_rate, iter=epoch, max_iter=args.num_epochs)
    lr_adv = poly_lr_scheduler(optimizerADV, args.learning_rate_adversarial, iter=epoch, max_iter=args.num_epochs)
    tq = tqdm(total=len(dataloader_source) * args.batch_size)
    tq.set_description('epoch %d, lr %f' % (epoch, args.learning_rate))


    for i, (data, label) in dataloader_source_iter:
      optimizerSEG.zero_grad()
      optimizerADV.zero_grad()
      
      for param in modelADV.parameters():
        param.requires_grad = False
    
      try:
        _, batch_target = dataloader_target_iter.__next__()
      except:
        dataloader_target_iter = enumerate(dataloader_target)
        _, batch_target = dataloader_target_iter.__next__()

    
      images_source = data.cuda()
      labels_source = label.long().cuda()
      images_target = batch_target
      images_target = images_target.cuda()


      #Computing the losses from the output of the seg net
      with amp.autocast(): 
        output_source_seg_seg, output_sup1_source_seg, output_sup2_source_seg = modelSEG(images_source)
        
        loss1_source_seg = loss_seg_source(output_source_seg_seg, labels_source)
        loss2_source_seg = loss_seg_source(output_sup1_source_seg, labels_source)
        loss3_source_seg = loss_seg_source(output_sup2_source_seg, labels_source)
        loss_source_seg = loss1_source_seg + loss2_source_seg + loss3_source_seg

        output_target_seg_seg, _, _ = modelSEG(images_target)
        output_target_adv_seg = modelADV(F.softmax(output_target_seg_seg))
        loss_buffer_target_seg = loss_adv_target_source(output_target_adv_seg, Variable(torch.FloatTensor(output_target_adv_seg.data.size()).fill_(source_label)).cuda())
        loss_target_seg = args.lambda_adv_target * loss_buffer_target_seg
        
        loss_tot = loss_source_seg + loss_target_seg
        scaler.scale(loss_tot).backward()

      
      ###Train the adversarial net
      #Start to accumulate grads in D while training the adversarial network
      for param in modelADV.parameters():
        param.requires_grad = True

      ##Train adversarial network with source dataset
      #detatching the output_source from the segnet
      output_source_seg_seg = output_source_seg_seg.detach()
      output_target_seg_seg = output_target_seg_seg.detach()

      #computing the loss of the adv net from source
      with amp.autocast():
        output_source_adv = modelADV(F.softmax(output_source_seg_seg))
        loss_D_source = loss_D(output_source_adv,Variable(torch.FloatTensor(output_source_adv.data.size()).fill_(source_label)).cuda())
        loss_D_source = loss_D_source/2


      #computing the loss of the adv net from target
      with amp.autocast():
        output_target_adv = modelADV(F.softmax(output_target_seg_seg))
        loss_D_target = loss_D(output_target_adv,Variable(torch.FloatTensor(output_target_adv.data.size()).fill_(target_label)).cuda())
        loss_D_target = loss_D_target/2

        #Backpropagating the loss
    
        scaler.scale(loss_D_target + loss_D_source).backward()
      


      #Make the itearation with the optmizer
      scaler.step(optimizerADV)
      scaler.step(optimizerSEG)
      scaler.update()

      tq.update(args.batch_size)

    epoch += 1

    
    #Saving Checkpoint
    if (epoch % args.checkpoint_step) == 0:

      combination_parameters1 = args.context_path_model + "_" + "ConvType==" + args.ConvType +  "_" + args.loss_type + "_" +\
      "Epochs==" + str(args.num_epochs) + "_" + \
      "UseLossWeights==" + str(args.weights_loss) + "_" +\
      "LRSEG==" + str(args.learning_rate) + "_" + "LRADV==" + str(args.learning_rate_adversarial) + "_" +\
      "LambdaADV==" + str(args.lambda_adv_target) + "_" + "BatchSize==" + str(args.batch_size) 


      EPOCH = epoch
      PATH_SEG = args.checkpoint_directory + "/SEG/" + combination_parameters1 + "_" + "modelSEG.pt"
      PATH_ADV = args.checkpoint_directory + "/ADV/" + combination_parameters1 + "_" + "modelADV.pt"
      
      

      torch.save({
          'epoch': EPOCH,
          'model_state_dict': modelSEG.state_dict(),
          'optimizer_state_dict': optmizer_segmentation.state_dict(),
          'LRSeg': args.learning_rate,
          }, PATH_SEG)
      
      torch.save({
          'epoch': EPOCH,
          'model_state_dict': modelADV.state_dict(),
          'optimizer_state_dict': optmizer_adversarial.state_dict(),
          'LRAdv': args.learning_rate,
          'LambdaADVTarget': args.lambda_adv_target
          }, PATH_ADV)





In [13]:
def main(params):
  #Use it to compute the total execution time
  import time
  start = time.time()

  #Setting all the standard values for the arguments
  parser = argparse.ArgumentParser()

  parser.add_argument('--cityscapes_dir', type=str,
                      default=(directory + "/dataset/Cityscapes"), 
                      help='Directory of the cityscapes dataset')
  parser.add_argument('--gta5_dir', type=str,
                      default=(directory+"/dataset/GTA5"), 
                      help='Directory of the GTA5 dataset')
  
  parser.add_argument('--image_dimensions', type=tuple,
                        default=(512,1024), 
                        help='Dimensions of images')

  parser.add_argument('--num_epochs', type=int, default=50  , help='Number of epochs to train for')
  parser.add_argument('--analysisMode', type=str, default="False"  , help='activate or not analysis mode')
  parser.add_argument('--checkpoint_directory', type=str, default = (directory + "/checkpoints") , help='Directory of the checkpoints folder')
  parser.add_argument('--val_results_directory', type=str, default = (directory + "/Validations") , help='Directory of the Validations folder')
  parser.add_argument('--checkpoint_step', type=int, default=1, help='How often to save checkpoints (epochs)')
  parser.add_argument('--number_steps', type=int, default=175, help='')
  parser.add_argument('--weights_loss', type=str, default="True", help='Use or not the weights')
  parser.add_argument('--weights_loss_dir', type=str, default= directory + "/save loss weights", help='Directory to save loss weigths')
  parser.add_argument('--loss_type', type=str, default="CrossEntropy", help='Choose type of loss')
  parser.add_argument('--ConvType', type=str, default="NSEP", help='Choose type of Convolution')

  
  parser.add_argument('--batch_size', type=int, default=8, help='Number of images in each batch')
  parser.add_argument('--context_path_model', type=str, default="resnet101",
                      help='The context path model you are using, resnet18, resnet101.')
  parser.add_argument('--learning_rate', type=float, default=0.00025, help='learning rate used for train')
  parser.add_argument('--learning_rate_adversarial', type=float, default=0.00025, help='learning rate used for train de adersarial network')
  parser.add_argument("--lambda_adv_target", type=float, default=0.001, help="lambda_adv for adversarial training.")
  parser.add_argument('--data', type=str, default='', help='path of training data')
  parser.add_argument('--num_workers', type=int, default=4, help='num of workers')
  parser.add_argument('--num_classes', type=int, default=19, help='num of object classes (with void)')
  parser.add_argument('--cuda', type=str, default='0', help='GPU ids used for training')
  parser.add_argument('--use_gpu', type=bool, default=True, help='whether to user gpu for training')
  parser.add_argument('--pretrained_model_path', type=str, default=None, help='path to pretrained model')
  parser.add_argument('--save_model_path', type=str, default=None, help='path to save model')

  args = parser.parse_args(params)

  #loading data
  dataset_target = CityScapeDataSet(args.cityscapes_dir, os.path.join(args.cityscapes_dir, "images"), "train.txt", target_dimensions = args.image_dimensions)
  dataset_source = GTA5DataSet(args.gta5_dir, os.path.join(args.gta5_dir, "images"), os.path.join(args.gta5_dir, "labels"), "train.txt", image_dimensions = args.image_dimensions)
  dataset_val = CityScapeDataSet_val(args.cityscapes_dir, os.path.join(args.cityscapes_dir, "images"), os.path.join(args.cityscapes_dir, "labels"), "val.txt", image_dimensions = args.image_dimensions)
  print(len(dataset_target))

  dataloader_target = DataLoader(dataset_target, batch_size = args.batch_size, num_workers = args.num_workers, shuffle = True, drop_last= True, persistent_workers= True)
  dataloader_source = DataLoader(dataset_source, batch_size = args.batch_size, num_workers = args.num_workers, shuffle = True, drop_last= True, persistent_workers= True)
  dataloader_val = DataLoader(dataset_val, batch_size = 1, num_workers = args.num_workers, shuffle = True, drop_last= False)


  #initialize models
  os.environ['CUDA_VISIBLE_DEVICES'] = args.cuda

  #Setting the combination of parameters that will be used as the name to the saved checkpoints
  combination_parameters2 = args.context_path_model + "_" + "ConvType==" + args.ConvType +  "_" + args.loss_type + "_" +\
  "Epochs==" + str(args.num_epochs) + "_" + \
  "UseLossWeights==" + str(args.weights_loss) + "_" +\
  "LRSEG==" + str(args.learning_rate) + "_" + "LRADV==" + str(args.learning_rate_adversarial) + "_" +\
  "LambdaADV==" + str(args.lambda_adv_target) + "_" + "BatchSize==" + str(args.batch_size)

  print("combParam2 = ",combination_parameters2)

  #Defining if the discriminator will be composed of Separable convolutions or standard convolutions
  modelSEG = BiSeNet(args.num_classes, args.context_path_model)
  if args.ConvType == "NSEP":
    modelADV = Discriminator(args.num_classes)
  elif args.ConvType == "SEP":
    modelADV = DiscriminatorSEP(args.num_classes)

  
  if torch.cuda.is_available() and args.use_gpu and args.analysisMode == "False":
      modelSEG = torch.nn.DataParallel(modelSEG).cuda()
      modelADV = torch.nn.DataParallel(modelADV).cuda()
  
  #Here we have to to define the optmizers
  optmizer_segmentation = torch.optim.SGD(modelSEG.parameters(), args.learning_rate, momentum=0.9, weight_decay=1e-4)
  optmizer_adversarial = torch.optim.Adam(modelADV.parameters(), lr=args.learning_rate_adversarial, betas=(0.9, 0.99))

  #Find out the parameters of the models
  #Analysis mode is the parameters that decides if the program will either run normally or just compute the metrics of the model
  if args.analysisMode == "True":
    import sys
    number_parametersSEG = sum(param.numel() for param in modelSEG.parameters())
    number_parametersADV = sum(param.numel() for param in modelADV.parameters())
    print("Num Param SEG: ", number_parametersSEG)
    print("Num Param ADV: ", number_parametersADV)
    
    label = torch.randn((1, 19, 1024, 512))
    macs, _ = profile(modelADV, inputs=(label, ))
    print("Floating point operations for Adversarial network is approximately: ",2*macs )
    print("Name: ", args.context_path_model + "_" + "ConvType==" + args.ConvType)
    sys.exit()



  #Import checkpoints 
  #If you define false it will just overwrite the current checkpoints or create new checkpoints
  use_checkpoints = False
  if use_checkpoints:
    checkpoint_seg = torch.load(args.checkpoint_directory + "/SEG/" + combination_parameters2 + "_" + "modelSEG.pt")
    checkpoint_adv = torch.load(args.checkpoint_directory + "/ADV/" + combination_parameters2 + "_" + "modelADV.pt")
  else:
    checkpoint_seg = None
    checkpoint_adv = None

  #Doing the calculation for the number of iterations
  size_iterations = len(dataloader_source.dataset)/args.batch_size
  params_update = ['--number_steps', str(int(size_iterations))]
  params = params + params_update
  args = parser.parse_args(params)

  #Calling the training function
  train_model(args, modelSEG, modelADV, optmizer_segmentation, optmizer_adversarial, dataloader_target, dataloader_source,checkpoint_seg, checkpoint_adv)
  #Calling the validation function
  precision, MIOU = validation_segmentation(args, modelSEG, dataloader_val, combination_parameters2)

  end = time.time()
  elapsed = end - start
  print("Elapsed Time :", elapsed, " Seconds")
  #Print the total elapsed time and finish the program
  


In [ ]:
if __name__ == '__main__':
  #This section was used for testing with different parameters
  #You can use these lists to train the model with multiple configurations without having to restart the program
  #Saves already all the validation results and all the checkpoints
  list_networks = list(["resnet101"])
  list_useWeights = list(["False"])
  list_convType = list(["NSEP"])
  epochs_size = list(["100"])
  list_losses = list(["CrossEntropy"])

  for model in list_networks:
    for use in list_useWeights:
      for typeC in list_convType:
        for ep in epochs_size:
          for loss in list_losses:
            #Empties the GPU memory and sets the arguments
            torch.cuda.empty_cache()
            if model == "resnet101":
              params = ["--context_path_model", model,
                        "--weights_loss", use,
                        "--batch_size", '4', #With more than 4 the GPU runs out of memory
                        "--ConvType", typeC,
                        "--num_epochs", ep,
                        '--loss_type',loss,
                        '--analysisMode', 'False',]
            else:
              params = ["--context_path_model", model,
                        "--weights_loss", use,
                        "--batch_size", '8',
                        "--ConvType", typeC,
                        "--num_epochs", ep,
                        '--loss_type',loss,
                        '--analysisMode', 'False',]
           
            main(params)
            